Pass or fail based on critieria 
final grade prediction for values that do not have a final grade 
a knn (idk which one yet)

In [228]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pylab as plt
from sklearn.metrics import accuracy_score
from sklearn import linear_model

from sklearn.neighbors import KNeighborsClassifier

In [229]:
student_performace = pd.read_csv('student_performance_updated_1000.csv')


In [230]:
def dataCleaning(student_performace):
    student_performace.columns = [s.strip().replace(' ','_') for s in student_performace.columns]
    return student_performace[
    (student_performace['StudyHoursPerWeek'] > 0) &
    (student_performace['AttendanceRate'] < 100) &
    (student_performace['StudyHoursPerWeek'] < (24 * 7)) &
    (student_performace['FinalGrade'] <= 100)|
    (student_performace['FinalGrade'].isna())&
    (student_performace['Study_Hours']>=0)
    ]


In [231]:
finalsGradePredictor = []
noData=student_performace['FinalGrade'].isna()
finalsGradePredictor = dataCleaning(student_performace[student_performace['FinalGrade'].isna()])
columns_to_check = finalsGradePredictor.columns.difference(['FinalGrade'])
finalsGradePredictor = finalsGradePredictor.dropna(subset=columns_to_check)
finalsGradePredictor

,StudentID,Name,Gender,AttendanceRate,StudyHoursPerWeek,PreviousGrade,ExtracurricularActivities,ParentalSupport,FinalGrade,Study_Hours,Attendance_(%),Online_Classes_Taken
5,6.0,Olivia,Female,95.0,30.0,88.0,1.0,High,NaN,2.8,97.0,False
32,2604.0,Yolanda Chen,Male,82.0,15.0,85.0,3.0,High,NaN,2.4,63.0,False
85,8381.0,Peter Welch,Female,88.0,15.0,90.0,2.0,Low,NaN,3.5,79.0,True
120,4394.0,John Warner,Male,92.0,30.0,65.0,3.0,High,NaN,4.3,59.0,True
124,1996.0,Manuel Baker,Female,70.0,8.0,85.0,1.0,Low,NaN,0.4,55.0,False
141,6562.0,Amy Rodriguez,Male,95.0,10.0,85.0,2.0,Low,NaN,2.7,62.0,False
222,5703.0,Alexis Walton,Female,78.0,10.0,60.0,1.0,Medium,NaN,4.3,74.0,False
225,5516.0,Jean Moreno,Male,90.0,15.0,60.0,3.0,High,NaN,1.0,56.0,True
260,1440.0,Cassandra Snyder,Female,92.0,17.0,82.0,2.0,Medium,NaN,0.7,69.0,True
335,1020.0,Timothy Clements,Male,70.0,17.0,60.0,0.0,Medium,NaN,4.4,72.0,False


In [232]:

#descriptive statistics

reduced_performace = student_performace.dropna()
reduced_performace = dataCleaning(reduced_performace)
x = reduced_performace.drop(columns=['FinalGrade'])
y = reduced_performace['FinalGrade']
reduced_performace.head()


,StudentID,Name,Gender,AttendanceRate,StudyHoursPerWeek,PreviousGrade,ExtracurricularActivities,ParentalSupport,FinalGrade,Study_Hours,Attendance_(%),Online_Classes_Taken
0,1.0,John,Male,85.0,15.0,78.0,1.0,High,80.0,4.8,59.0,False
1,2.0,Sarah,Female,90.0,20.0,85.0,2.0,Medium,87.0,2.2,70.0,True
2,3.0,Alex,Male,78.0,10.0,65.0,0.0,Low,68.0,4.6,92.0,False
3,4.0,Michael,Male,92.0,25.0,90.0,3.0,High,92.0,2.9,96.0,False
6,7.0,Daniel,Male,70.0,8.0,60.0,0.0,Low,62.0,4.5,96.0,False


In [233]:
reduced_performace.describe()

,StudentID,AttendanceRate,StudyHoursPerWeek,PreviousGrade,ExtracurricularActivities,FinalGrade,Study_Hours,Attendance_(%)
count,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000
mean,5419.272868,85.378295,17.682171,77.931783,1.536434,80.122481,2.426512,77.052713
std,2682.454600,7.282423,6.355641,10.060917,1.041375,9.609561,1.578680,18.245148
min,1.000000,70.000000,8.000000,60.000000,0.000000,62.000000,-5.000000,50.000000
25%,3044.000000,82.000000,12.000000,70.000000,1.000000,72.000000,1.200000,63.000000
50%,5425.000000,85.000000,18.000000,82.000000,2.000000,80.000000,2.500000,76.000000
75%,7812.000000,91.000000,22.000000,86.000000,2.000000,88.000000,3.700000,89.000000
max,9991.000000,95.000000,30.000000,90.000000,3.000000,92.000000,5.000000,200.000000


We want to predict the final grades of students based on the various supervised learning techniques we have done in class so far
these are 
knn
nn
linear regression 
regression trees
ensembles

## Linear Regression(multiple)

In [234]:
reduced_performace= pd.get_dummies(reduced_performace,drop_first=True,columns=['ParentalSupport','Gender'])

In [235]:
#define dependent and independent variables
variables =['StudyHoursPerWeek', 'AttendanceRate', 'PreviousGrade','Attendance_(%)','ExtracurricularActivities','Study_Hours'
            ,'Online_Classes_Taken','ParentalSupport_Low','ParentalSupport_Medium','Gender_Male']
X=reduced_performace[variables]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
y=reduced_performace[['FinalGrade']]
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
finalsGradePredictor= pd.get_dummies(finalsGradePredictor,drop_first=True,columns=['ParentalSupport','Gender'])
predictedFinal = regr.predict(finalsGradePredictor[variables])
predictedFinal

array([80.82029439, 78.28044918, 81.12543119, 79.34740941, 78.77360451,
       78.57359261, 81.50869521, 78.02001475, 80.31003354, 79.86198363,
       80.97319726, 80.45468816, 81.73353618, 81.5078606 , 80.88864926,
       80.86788637, 78.24787608, 81.54905871, 78.15619635, 80.19089172,
       80.85775999, 79.76310279, 81.4174237 ])

## Knn

In [236]:
reduced_performace = student_performace.dropna()
reduced_performace = dataCleaning(reduced_performace)
x = reduced_performace.drop(columns=['FinalGrade'])
y = reduced_performace['FinalGrade']
reduced_performace= pd.get_dummies(reduced_performace,drop_first=False,columns=['ParentalSupport','Gender'])

In [237]:

scaler = StandardScaler()
normal = scaler.fit_transform(X[variables])
X_train, X_test, y_train, y_test = train_test_split(normal, y, test_size=0.3, random_state=42)
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

,n_neighbors,1
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [238]:
y_pred = knn.predict(X_test)
print(f"Test Accuracy (k=5): {accuracy_score(y_test, y_pred):.2f}")

Test Accuracy (k=5): 0.10
